In [ ]:
!pip install -q scikit-learn sentence-transformers transformers torch numpy accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 10.2 MB/s eta 0:00:00


In [ ]:
%%writefile travel_planning_qwen25.py

"""
Travel Planning System with Multi-Agent Architecture

This script defines multiple specialized agents, each designed for a specific aspect of travel planning:

1. WeatherAnalysisAgent:
   - Predicts the best months to visit a location using historical weather data.
   - Uses a Random Forest regression model for predictions.

2. HotelRecommenderAgent:
   - Recommends hotels based on semantic similarity between user preferences and hotel descriptions.
   - Utilizes SentenceTransformer embeddings for semantic comparison.

3. ItineraryPlannerAgent:
   - Generates detailed travel itineraries using a text-generation language model.
   - Leverages the Qwen2.5-3B-Instruct model for generating realistic itineraries based on provided prompts.

4. SummaryAgent:
   - Summarizes the travel plan details into a concise, client-friendly email.
   - Estimates the total cost of the trip and uses Qwen2.5-3B-Instruct to generate the summary content.

These agents collectively enable an automated and intelligent approach to personalized travel planning.
"""
from typing import List, Dict
import numpy as np
import torch
from sklearn.ensemble import RandomForestRegressor
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


class WeatherAnalysisAgent:
    """
    An agent that analyzes historical weather data to predict optimal travel periods.

    Attributes:
        model (RandomForestRegressor): A random forest model trained on historical weather data.
    """
    def __init__(self):
        # Uses RandomForest for weather predictions based on historical data
        self.model = RandomForestRegressor(n_estimators=100)

    def train(self, historical_data: Dict):
        # Training on historical weather data
        X = np.array([[d['month'], d['latitude'], d['longitude']] for d in historical_data])
        y = np.array([d['weather_score'] for d in historical_data])
        self.model.fit(X, y)

    def predict_best_time(self, location: Dict) -> Dict:
        # Predicts the best time to visit a location based on weather patterns
        predictions = []
        for month in range(1, 13):
            # predict returns a 2D array, we take the first (and only) element
            prediction = self.model.predict([[
                month,
                location['latitude'],
                location['longitude']
            ]]).item()  # .item() converts numpy array to scalar
            predictions.append({'month': month, 'score': float(prediction)})

        return {
            'best_months': sorted(predictions, key=lambda x: x['score'], reverse=True)[:3],
            'location': location
        }


class HotelRecommenderAgent:
    """
    An agent for recommending hotels that best match user preferences using semantic embeddings.

    Attributes:
        encoder (SentenceTransformer): Sentence embedding model for semantic matching.
        hotels_db (List[Dict]): Database of hotel information.
        hotels_embeddings (np.ndarray): Precomputed embeddings of hotel descriptions.
    """
    def __init__(self):
        # Uses SentenceTransformer for hotel description embeddings
        self.encoder = SentenceTransformer('all-MiniLM-L6-v2')
        self.hotels_db = []
        self.hotels_embeddings = None

    def add_hotels(self, hotels: List[Dict]):
        # Updates the hotel database and computes embeddings
        self.hotels_db = hotels
        descriptions = [h['description'] for h in hotels]
        self.hotels_embeddings = self.encoder.encode(descriptions)

    def find_hotels(self, preferences: str, top_k: int = 5) -> List[Dict]:
        # Finds hotels most similar to preferences using semantic similarity
        pref_embedding = self.encoder.encode([preferences])
        similarities = np.dot(self.hotels_embeddings, pref_embedding.T).flatten()

        top_indices = similarities.argsort()[-top_k:][::-1]
        return [
            {**self.hotels_db[i], 'similarity_score': float(similarities[i])}
            for i in top_indices
        ]


class ItineraryPlannerAgent:
    """
    An agent that creates travel itineraries by generating descriptive text using Qwen2.5-3B.

    Attributes:
        tokenizer: Qwen2.5 tokenizer
        model: Qwen2.5-3B-Instruct model with 4-bit quantization
    """
    def __init__(self):
        print("Qwen2.5-3B 모델 로딩 중...")

        # 4-bit quantization config for T4 GPU memory optimization
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )

        model_name = "Qwen/Qwen2.5-3B-Instruct"

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            device_map="auto",
            trust_remote_code=True
        )

        print("모델 로딩 완료!")

    def create_itinerary(self, destination_info: Dict, weather_info: Dict,
                         hotel_info: Dict, duration: int) -> Dict:
        prompt = self._create_prompt(destination_info, weather_info, hotel_info, duration)

        messages = [
            {"role": "system", "content": "당신은 전문 여행 플래너입니다. 구체적이고 실용적인 여행 일정을 작성해주세요."},
            {"role": "user", "content": prompt}
        ]

        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)

        # Generate the itinerary
        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=400,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return {
            'itinerary': response,
            'duration': duration,
            'destination': destination_info['name']
        }

    def _create_prompt(self, destination_info: Dict, weather_info: Dict,
                       hotel_info: Dict, duration: int) -> str:
        return f"""다음 정보를 바탕으로 {duration}일간의 상세한 여행 일정을 작성해주세요:

목적지: {destination_info['name']}
최적 방문 시기: {weather_info['best_months'][0]['month']}월
숙박: {hotel_info[0]['name']}
주요 관광지: {', '.join(destination_info['attractions'])}

각 날짜별로 아침, 점심, 저녁 일정을 포함하여 구체적으로 작성해주세요."""


class SummaryAgent:
    """
    An agent that generates summarized travel plans in the form of personalized emails.

    Attributes:
        tokenizer: Qwen2.5 tokenizer
        model: Qwen2.5-3B-Instruct model with 4-bit quantization for email creation
    """
    def __init__(self):
        print("Qwen2.5-3B 모델 로딩 중... (SummaryAgent)")

        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )

        model_name = "Qwen/Qwen2.5-3B-Instruct"

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            device_map="auto",
            trust_remote_code=True
        )

        print("모델 로딩 완료!")

    def calculate_total_price(self, hotel_info: Dict, duration: int) -> float:
        # Calculate total trip price
        hotel_cost = hotel_info[0]['price'] * duration
        # Estimate additional costs (activities, meals, transport)
        daily_expenses = 100  # Simplified example
        additional_costs = daily_expenses * duration

        return hotel_cost + additional_costs

    def create_email(self, trip_data: Dict, client_name: str) -> Dict:
        total_price = self.calculate_total_price(
            trip_data['recommended_hotels'],
            trip_data['itinerary']['duration']
        )

        prompt = f"""다음 여행 계획 정보를 바탕으로 고객에게 보낼 전문적인 이메일을 작성해주세요:

고객명: {client_name}
목적지: {trip_data['itinerary']['destination']}
기간: {trip_data['itinerary']['duration']}일
최적 방문 시기: {trip_data['weather_analysis']['best_months'][0]['month']}월
추천 호텔: {trip_data['recommended_hotels'][0]['name']}
예상 총 비용: ${total_price}

여행 일정:
{trip_data['itinerary']['itinerary']}

정중하고 전문적인 톤으로 작성해주세요."""

        messages = [
            {"role": "system", "content": "당신은 전문 여행사 직원입니다. 고객에게 보낼 정중하고 상세한 이메일을 작성해주세요."},
            {"role": "user", "content": prompt}
        ]

        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)

        # Generate email using LLM
        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=600,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return {
            'email_content': response,
            'total_price': total_price,
            'summary_data': {
                'destination': trip_data['itinerary']['destination'],
                'duration': trip_data['itinerary']['duration'],
                'hotel': trip_data['recommended_hotels'][0]['name'],
                'best_month': trip_data['weather_analysis']['best_months'][0]['month']
            }
        }


class TravelPlanningSystem:
    def __init__(self):
        self.weather_agent = WeatherAnalysisAgent()
        self.hotel_agent = HotelRecommenderAgent()
        self.itinerary_agent = ItineraryPlannerAgent()
        self.summary_agent = SummaryAgent()

    def setup(self, historical_weather_data: Dict, hotels_database: List[Dict]):
        # Initialize and train the models
        self.weather_agent.train(historical_weather_data)
        self.hotel_agent.add_hotels(hotels_database)

    def plan_trip(self, destination: Dict, preferences: str, duration: int, client_name: str) -> Dict:
        # 1. Weather analysis and best time prediction
        weather_analysis = self.weather_agent.predict_best_time(destination)

        # 2. Hotel search
        recommended_hotels = self.hotel_agent.find_hotels(preferences)

        # 3. Itinerary creation
        itinerary = self.itinerary_agent.create_itinerary(
            destination,
            weather_analysis,
            recommended_hotels,
            duration
        )

        # 4. Create summary email and calculate price
        trip_data = {
            'weather_analysis': weather_analysis,
            'recommended_hotels': recommended_hotels,
            'itinerary': itinerary
        }

        summary = self.summary_agent.create_email(trip_data, client_name)

        return {
            **trip_data,
            'summary': summary
        }


def main():
    # Example data with a full year of weather information
    historical_weather_data = [
        {'month': 1, 'latitude': 41.9028, 'longitude': 12.4964, 'weather_score': 0.5},
        {'month': 2, 'latitude': 41.9028, 'longitude': 12.4964, 'weather_score': 0.6},
        {'month': 3, 'latitude': 41.9028, 'longitude': 12.4964, 'weather_score': 0.7},
        {'month': 4, 'latitude': 41.9028, 'longitude': 12.4964, 'weather_score': 0.8},
        {'month': 5, 'latitude': 41.9028, 'longitude': 12.4964, 'weather_score': 0.85},
        {'month': 6, 'latitude': 41.9028, 'longitude': 12.4964, 'weather_score': 0.9},
        {'month': 7, 'latitude': 41.9028, 'longitude': 12.4964, 'weather_score': 0.95},
        {'month': 8, 'latitude': 41.9028, 'longitude': 12.4964, 'weather_score': 0.9},
        {'month': 9, 'latitude': 41.9028, 'longitude': 12.4964, 'weather_score': 0.85},
        {'month': 10, 'latitude': 41.9028, 'longitude': 12.4964, 'weather_score': 0.7},
        {'month': 11, 'latitude': 41.9028, 'longitude': 12.4964, 'weather_score': 0.6},
        {'month': 12, 'latitude': 41.9028, 'longitude': 12.4964, 'weather_score': 0.5},
    ]

    # Sample hotel database
    hotels_database = [
        {
            'name': 'Grand Hotel',
            'description': 'Luxury hotel in city center with spa and restaurant',
            'price': 300
        },
        {
            'name': 'Boutique Resort',
            'description': 'Intimate boutique hotel with personalized service',
            'price': 250
        },
        {
            'name': 'City View Hotel',
            'description': 'Modern hotel with panoramic city views',
            'price': 200
        }
    ]

    # Initialize the system
    system = TravelPlanningSystem()
    system.setup(historical_weather_data, hotels_database)

    # Plan a trip
    destination = {
        'name': 'Rome',
        'latitude': 41.9028,
        'longitude': 12.4964,
        'attractions': ['Colosseum', 'Vatican', 'Trevi Fountain']
    }

    preferences = """Looking for a luxury hotel in the city center,
    preferably with spa facilities and fine dining options"""

    client_name = "John Smith"

    # Generate trip plan
    trip_plan = system.plan_trip(destination, preferences, duration=3, client_name=client_name)

    # Print results in a readable format
    print("\nTRAVEL PLANNING RESULTS:")
    print("-" * 50)
    print(f"Client: {client_name}")
    print(f"Destination: {destination['name']}")
    print("\nGenerated Email:")
    print("-" * 20)
    print(trip_plan['summary']['email_content'])
    print("\nEstimated Total Price:")
    print(f"${trip_plan['summary']['total_price']}")


if __name__ == "__main__":
    main()

Writing travel_planning_qwen25.py


In [ ]:
%ls -al

total 32
drwxr-xr-x 1 root root  4096 Oct 12 15:22 ./
drwxr-xr-x 1 root root  4096 Oct 12 15:20 ../
drwxr-xr-x 4 root root  4096 Oct  9 13:35 .config/
drwxr-xr-x 1 root root  4096 Oct  9 13:36 sample_data/
-rw-r--r-- 1 root root 14762 Oct 12 15:22 travel_planning_qwen25.py


In [ ]:
!python travel_planning_qwen25.py

2025-10-12 15:23:39.163411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760282619.195550     968 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760282619.205533     968 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760282619.230719     968 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760282619.230757     968 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760282619.230765     968 computation_placer.cc:177] computation placer alr